In [1]:
from uniparse import Vocabulary
from uniparse.dataprovider import batch_by_buckets
import random
from collections import defaultdict
import numpy as np
from sklearn.utils import class_weight
from collections import defaultdict
import numpy as np
from uniparse import Model, Vocabulary

from uniparse.callbacks import ModelSaveCallback
from uniparse.dataprovider import batch_by_buckets

In [2]:
vocab = Vocabulary()
samples = vocab._read_conll("../data/ptb_conllu/train.conllu", tokenize=False)

In [18]:
from collections import Counter
import pandas as pd

def count_3rd_order_sets(treebank_file, vocab):
    samples = vocab._read_conll(treebank_file, tokenize=False)
    counter = Counter()
    for sample in samples:
        extract_3O_paths(sample[3], sample[4], counter)
        
    factors = [{"afactor":"%s->%s->%s"%k, "count":v} for k,v in counter.items()]
    df = pd.DataFrame.from_dict(factors).sort_values("count", ascending=False)
    
    return df

def count_rel_labels(treebank_file, vocab):
    samples = vocab._read_conll(treebank_file, tokenize=False)
    counter = Counter()
    for sample in samples:
        counter.update(sample[4])
        
    factors = [{"afactor":k, "count":v} for k,v in counter.items()]
    df = pd.DataFrame.from_dict(factors).sort_values("count", ascending=False)
    
    return df

def extract_3O_paths(tree, tags, counter, root_token="<root>"):    
    for m, h in enumerate(tree):
        p = tree[h]
        parent_tag, head_tag, modifier_tag = [tags[i] if i >= 0 else tags[0] for i in [p, h, m]]
        parent_tag, head_tag, modifier_tag = [t if t != 1 else root_token for t in [parent_tag, head_tag, modifier_tag]]
        counter.update([(parent_tag, head_tag, modifier_tag)])

def factor_to_str(t):
    return "->".join(t)

def get_factor_weights(dataframe):
    dataset = []
    factor_map = defaultdict(int)
    for i, row in dataframe.iterrows():
        factor = row["afactor"]
        factor_map[i] = factor
        for _ in range(row["count"]):
            dataset.append(i)
    classes = np.unique(dataset)
    weights = class_weight.compute_class_weight("balanced", classes, dataset)
    return {factor_map[c]:w for c,w in zip(classes,weights)}


In [19]:
count_rel_labels("../data/en_ewt-ud-train.conllu", vocab)

,afactor,count
2,punct,23596
10,case,17442
5,nsubj,16258
8,det,15753
1,root,12543
0,1,12543
24,advmod,10956
7,obj,10189
11,obl,9152
4,amod,9066
